In [1]:
# Created by Emille Ishida on 03/09/2025

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
from copy import deepcopy

In [2]:
# read results of xmatch
data = pd.read_parquet('/media3/CRP8/TDE/data/xmatch/galactic/')

In [3]:
data.shape

(3295444, 10)

In [4]:
data_obj = data.groupby('objectId', sort=False, as_index=False).agg(list)

In [5]:
data_obj.shape

(88594, 10)